In [1]:
from config import DATABASE_URL
from src.utils.database_connection_utils import get_database_session

In [2]:
session, engine = get_database_session(DATABASE_URL)


In [3]:
sql_query = "select * from article"

In [4]:
import pandas as pd
import numpy as np

In [5]:
data = None
with engine.connect() as connection:
    data = pd.read_sql_query(sql=sql_query, con=connection)

In [6]:
data.shape

(11218, 9)

In [7]:
data.website_origin.unique()

array(['https://forumdesas.net/', 'https://www.7sur7.cd',
       'https://www.politico.cd', 'https://www.actualite.cd',
       'https://www.mediacongo.net'], dtype=object)

In [8]:
data.head()

,id,title,content,summary,posted_at,website_origin,url,author,saved_at
0,1,La Commission tarifaire propose des mesures d’...,Les membres de la Commission tarifaire viennen...,None,2022-09-05,https://forumdesas.net/,https://www.forumdesas.net/2022/05/la-commissi...,Kléber KUNGU,2022-05-12 16:53:56.464801+00:00
1,2,La Commission tarifaire pour la mise en oeuvre...,Les membres de la Commission tarifaire so...,None,2022-04-05,https://forumdesas.net/,https://www.forumdesas.net/2022/05/la-commissi...,Kléber KUNGU,2022-05-12 16:53:56.505433+00:00
2,3,Lancement du 1er Laboratoire de recherche et d...,Vodacom Congo vient de signer un partenariat a...,None,2022-04-23,https://forumdesas.net/,https://www.forumdesas.net/2022/04/lancement-d...,None,2022-05-12 16:53:56.548859+00:00
3,4,Léopards : Hectór Cúper ce mardi à Kinshasa,"Le sélectionneur des Léopards de la RDC, Hectó...",None,2022-03-05,https://forumdesas.net/,https://www.forumdesas.net/2022/05/leopards-he...,Gloire BATOMENE,2022-05-12 16:53:56.561548+00:00
4,5,» Air Congo » en voie d’être créée pour s’en...,Le protocole d’accord était déjà signé entre l...,None,2022-11-05,https://forumdesas.net/,https://www.forumdesas.net/2022/05/air-congo-e...,KANDOLO M.,2022-05-12 16:53:56.682724+00:00


In [9]:
random_index = np.random.randint(0, data.shape[0])
sample_doc = data.content[random_index]
sample_doc

"L'État-major, commandement du 3310ième régiment des Forces Armées de la République Démocratique du Congo a, dans un communiqué officiel parvenu à 7SUR7.CD ce vendredi 15 mai 2020, annoncé l'identification des militaires et des officiers gradés qui sont basés dans differents quartiers de la ville commerciale de Butembo (Nord-Kivu).L'armée congolaise demande aux habitants de Butembo de fournir, dans leur environnement immédiat, l'adresse de « tout militaire et officier gradé » résidant à la cité en vue d'une identification par les services spécialisés.« Il est demandé aux chefs des cellules, chefs de quartiers et à la société civile des communes, de bien vouloir recueillir toutes ces informations et de nous les faire parvenir par le canal des bourgmestres des communes pour la centralisation », écrit le lieutenant colonel Ombeni Murengisi, commandant des FARDC-Butembo.Il sied de préciser par ailleurs que plusieurs positions militaires sont déjà installées dans différents quartiers périph

How should I split the text , should I use the sentence or the word level to detect names in this entities?

I will try sentence and see how the french NER model from spacy will perform and then I will try more complicated approaches.

By the end of this work we should have a pipeline that takes the text and mask the name entities.

In [31]:
import re
from nltk.tokenize import sent_tokenize
from unicodedata import normalize

In [56]:
def remove_accents(input_str):
    nfkd_form = normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore').decode("utf-8")
    return only_ascii

def remplace_apostrophes(input_str):
    apostrophes = r"[\']"
    text = re.compile(input_str).sub("", text) # remove apostrophes

def clean_text(text):
    cleaned_text = remove_accents(text)
    return str(cleaned_text)

In [37]:
clean_text(sample_doc)

'Etat-major, commandement du 3310ieme regiment des Forces Armees de la Republique Democratique du Congo a, dans un communique officiel parvenu a 7SUR7.CD ce vendredi 15 mai 2020, annonce identification des militaires et des officiers grades qui sont bases dans differents quartiers de la ville commerciale de Butembo (Nord-Kivu).armee congolaise demande aux habitants de Butembo de fournir, dans leur environnement immediat, adresse de  tout militaire et officier grade  residant a la cite en vue une identification par les services specialises. Il est demande aux chefs des cellules, chefs de quartiers et a la societe civile des communes, de bien vouloir recueillir toutes ces informations et de nous les faire parvenir par le canal des bourgmestres des communes pour la centralisation , ecrit le lieutenant colonel Ombeni Murengisi, commandant des FARDC-Butembo.Il sied de preciser par ailleurs que plusieurs positions militaires sont deja installees dans differents quartiers peripheriques de But

In [38]:
tokenized_document = sent_tokenize(clean_text(sample_doc), language="french")

In [39]:
tokenized_document

['Etat-major, commandement du 3310ieme regiment des Forces Armees de la Republique Democratique du Congo a, dans un communique officiel parvenu a 7SUR7.CD ce vendredi 15 mai 2020, annonce identification des militaires et des officiers grades qui sont bases dans differents quartiers de la ville commerciale de Butembo (Nord-Kivu).armee congolaise demande aux habitants de Butembo de fournir, dans leur environnement immediat, adresse de  tout militaire et officier grade  residant a la cite en vue une identification par les services specialises.',
 'Il est demande aux chefs des cellules, chefs de quartiers et a la societe civile des communes, de bien vouloir recueillir toutes ces informations et de nous les faire parvenir par le canal des bourgmestres des communes pour la centralisation , ecrit le lieutenant colonel Ombeni Murengisi, commandant des FARDC-Butembo.Il sied de preciser par ailleurs que plusieurs positions militaires sont deja installees dans differents quartiers peripheriques d

In [40]:

import spacy

In [41]:
nlp = spacy.load("fr_core_news_sm")

In [42]:
from spacy import displacy

In [43]:
processed_sentence = nlp(tokenized_document[2])
processed_document = nlp(clean_text(sample_doc))

In [44]:
print(nlp.pipe_names)

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [45]:
tokenized_document[0]

'Etat-major, commandement du 3310ieme regiment des Forces Armees de la Republique Democratique du Congo a, dans un communique officiel parvenu a 7SUR7.CD ce vendredi 15 mai 2020, annonce identification des militaires et des officiers grades qui sont bases dans differents quartiers de la ville commerciale de Butembo (Nord-Kivu).armee congolaise demande aux habitants de Butembo de fournir, dans leur environnement immediat, adresse de  tout militaire et officier grade  residant a la cite en vue une identification par les services specialises.'

In [46]:
displacy.render(processed_sentence, style="ent", jupyter=True)

In [47]:
displacy.render(processed_document, style="ent", jupyter=True)

The spacy model seems to be do well in detecting names, organization  ,but it can be improve to detec orgainization and more place.

In [23]:
{"meta": 
    {"source": "BBC News","via":"News API"},
 "_task_hash":142354664,
 "text":"ViewRay (VRAY +1.2%) inks an agreement with an affiliate of Fosun International Limited for the direct sale of stock and warrants yielding ~$59.1M in gross proceeds.Specifically, the company will sell",
 "_input_hash":1622806810}

{'meta': {'source': 'BBC News', 'via': 'News API'},
 '_task_hash': 142354664,
 'text': 'ViewRay (VRAY +1.2%) inks an agreement with an affiliate of Fosun International Limited for the direct sale of stock and warrants yielding ~$59.1M in gross proceeds.Specifically, the company will sell',
 '_input_hash': 1622806810}

Seems like that the spacy model is mislead by accent and `" `, so let us remove them from the text.

And improve the spacy annodation , if by saturaday I can'tafind anything I will need to continue with what I have and make it a baseline

In [48]:
def tokenize_document(document):
    """given a document split the document into sentences and return a list of those sentence

    Args:
        document (_type_): _description_

    Returns:
        _type_: _description_
    """
    cleaned_document = clean_text(document)
    tokenized_document = sent_tokenize(cleaned_document, language="french")
    return tokenized_document

In [49]:
def generate_sentences(document_serie):
    """takea serie of documents and return a list of sentences

    Args:
        document_serie (_type_): _description_

    Returns:
        _type_: _description_
    """
    for document in document_serie:
        sentences = tokenize_document(document)
        for sentence in sentences:
            yield sentence

In [50]:
from secrets import token_hex
import json

In [54]:
def build_json_document(document_series, output_file_path):
    """take a series of documents and return a list of json documents

    Args:
        document_series (_type_): _description_

    Returns:
        _type_: _description_
    """
    with open(output_file_path, "w") as output_file:
        for sentence in generate_sentences(document_series):
            item = {"_task_hash": token_hex(6), "_input_hash": token_hex(6),  "text": sentence}
            output_file.write(json.dumps(item, ensure_ascii=False) + "\n")

In [52]:
from pathlib import Path



In [55]:
corpus_path = Path.cwd().joinpath("data", "corpus", "congo_news.jsonl")
build_json_document(data.content, corpus_path)

In [30]:
corpus_path.absolute().__str__()

'/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/corpus/congo_news.jsonl'

###  Setting up Model Tracking Experiments

In [57]:
import mlflow

mlflow.set_tracking_uri("sqllite//:mlflow.db")
mlflow.set_experiment("congo-news-ner-experiments")

2022/05/24 12:00:37 INFO mlflow.tracking.fluent: Experiment with name 'congo-news-ner-experiments' does not exist. Creating a new experiment.


<Experiment: artifact_location='sqllite//:mlflow.db/1', experiment_id='1', lifecycle_stage='active', name='congo-news-ner-experiments', tags={}>

In [ ]:
with mlflow.run():
    mlflow.set_tag("corpus", "congo_news")